# MachineHack Hackathon Predict the Price of Books

# 1. Exploring the dataset

In [4]:
import pandas as pd
import numpy as np
train = pd.read_excel("/home/abhishek/Documents/Participants_Data/Data_Train.xlsx")
test = pd.read_excel("/home/abhishek/Documents/Participants_Data/Data_Test.xlsx")

In [5]:
train.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [6]:
train.shape

(6237, 9)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 9 columns):
Title           6237 non-null object
Author          6237 non-null object
Edition         6237 non-null object
Reviews         6237 non-null object
Ratings         6237 non-null object
Synopsis        6237 non-null object
Genre           6237 non-null object
BookCategory    6237 non-null object
Price           6237 non-null float64
dtypes: float64(1), object(8)
memory usage: 438.6+ KB


In [8]:
train.describe(include='all').head(4)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
count,6237,6237,6237,6237,6237,6237,6237,6237,6237.0
unique,5568,3679,3370,36,342,5549,345,11,NaN
top,A Game of Thrones (A Song of Ice and Fire),Agatha Christie,"Paperback,– 5 Oct 2017",5.0 out of 5 stars,1 customer review,A Tinkle Double Digest is two Tinkle Digests i...,Action & Adventure (Books),Action & Adventure,NaN
freq,4,69,48,1375,1040,8,947,818,NaN


In [9]:
train.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price'],
      dtype='object')

In [10]:
# Removing Synopsis since here we are not going to use this feature
train.drop('Synopsis', axis=1, inplace=True)
test.drop('Synopsis', axis=1, inplace=True)

In [11]:
train.head(5)

,Title,Author,Edition,Reviews,Ratings,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography",965.62


In [12]:
train.isnull().sum()

Title           0
Author          0
Edition         0
Reviews         0
Ratings         0
Genre           0
BookCategory    0
Price           0
dtype: int64

# 2. Preprocessing the data

### Splitting Edition Column

In [13]:
def split_edition(data):
    edition = list(data)
    ed_type = [i.split(",– ")[0].strip().upper() for i in edition]
    edit_date = [i.split(",– ")[1].strip() for i in edition]
    m_y = [i.split()[-2:] for i in edit_date]
    
    for i in range(len(m_y)):
        if(len(m_y[i])) == 1:
            m_y[i].insert(0,'NA')
            
    months =  ['Apr','Aug','Dec','Feb', 'Jan', 'Jul','Jun','Mar','May','NA','Nov','Oct','Sep']
    ed_month = [m_y[i][0].upper() if m_y[i][0] in months else 'NA' for i in range(len(m_y))]
    ed_year = [int(m_y[i][1].strip()) if m_y[i][1].isdigit() else 0 for i in range(len(m_y))]
    
    return ed_type, ed_month, ed_year

### Splitting author column

In [14]:
# finding maximum number of author for a single book
author_1 = list(train['Author'])
author_2 = list(test['Author'])

author_1.extend(author_2)

authors = [i.split(",") for i in author_1]
max = 1
for i in authors:
    if len(i) >= max:
        max = len(i)
print("Max. number of authors for a single book = ", max)

for i in range(len(authors)):
    if len(authors[i]) == max:
        print(i)

all_authors = [author.strip().upper() for listin in authors for author in listin]

Max. number of authors for a single book =  7
7008


In [15]:
# splitting the author into 7 new columns
def split_column(data):
    authorscol = list(data)
    
    A1 = []
    A2 = []
    A3 = []
    A4 = []
    A5 = []
    A6 = []
    A7 = []
    
    for i in authorscol:
        try:
            A1.append(i.split(',')[0].strip().upper())
        except:
            A1.append('None')
            
        try:
            A2.append(i.split(',')[1].strip().upper())
        except:
            A2.append('None')
        
        try:
            A3.append(i.split(',')[2].strip().upper())
        except:
            A3.append('None')
         
        try:
            A4.append(i.split(',')[3].strip().upper())
        except:
            A4.append('None')
        
        try:
            A5.append(i.split(',')[4].strip().upper())
        except:
            A5.append('None')
        
        try:
            A6.append(i.split(',')[5].strip().upper())
        except:
            A6.append('None')
        
        try:
            A7.append(i.split(',')[6].strip().upper())
        except:
            A7.append('None')
            
    return A1,A2,A3,A4,A5,A6,A7

all_authors.append('None')

### Splitting Genre Column

In [16]:
# finding maximum no. of genres for a single book
genre_1 = list(train['Genre'])
genre_2 = list(test['Genre'])

genre_1.extend(genre_2)

genrelis = [i.split(",") for i in genre_1]
max = 1
for i in genrelis:
    if len(i) >= max:
        max = len(i)
print("Max. number of genres in a single book: ", max)
    
genre_all = [genre.strip().upper() for list1 in genrelis for genre in list1]

Max. number of genres in a single book:  2


In [17]:
# A method to split the Genre column in to 2 new columns
def split_genres(data):
    genrecol = list(data)
    
    G1 = []
    G2 = []
    
    for i in genrecol:
        try:
            G1.append(i.split(',')[0].strip().upper())
        except:
            G1.append('None')
            
        try:
            G2.append(i.split(',')[1].strip().upper())
        except:
            G2.append('None')
    return G1,G2

genre_all.append("None")

### Splitting BookCategory Column

In [18]:
# finding maximum no. of BookCategory for a single book
cat_1 = list(train['BookCategory'])
cat_2 = list(test['BookCategory'])

cat_1.extend(cat_2)

catlis = [i.split(",") for i in cat_1]
max = 1
for i in catlis:
    if len(i) >= max:
        max = len(i)
print("Max. number of categories for a single book: ", max)
    
cat_all = [cat.strip().upper() for list1 in catlis for cat in list1]

Max. number of categories for a single book:  2


In [19]:
# A method to split the BookCategory column in to 2 new columns
def split_category(data):
    catcol = list(data)
    
    C1 = []
    C2 = []
    
    for i in catcol:
        try:
            C1.append(i.split(',')[0].strip().upper())
        except:
            C1.append('None')
            
        try:
            C2.append(i.split(',')[1].strip().upper())
        except:
            C2.append('None')
    return C1,C2

cat_all.append("None")

### cleaning and creating a new dataset

In [20]:
import re
def restructure(data):
    
    #cleaning title
    titles = list(data['Title'])
    titles = [title.strip().upper() for title in titles]
    #cleaning author columns
    a1,a2,a3,a4,a5,a6,a7 = split_column(data['Author'])
    #cleaning genre column
    g1,g2 = split_genres(data['Genre'])
    #cleaning BookCategory Column
    c1,c2 = split_category(data['BookCategory'])
    #cleaning edition column
    ed_type, ed_month, ed_year = split_edition(data['Edition'])
    #cleaning rating and reviews column
    ratings = list(data['Reviews'])
    ratings = [float(re.sub(" out of 5 stars", "",i).strip()) for i in ratings]
    
    reviews = list(data['Ratings'])
    plu = ' customer reviews'
    reviews = [re.sub(" customer reviews", "",i) if plu in i else re.sub(" customer review", "",i) for i in reviews]
    reviews = [int(re.sub(",", "", i).strip()) for i in reviews]
     
    # Forming the Structured dataset
    structured_data = pd.DataFrame({'Title': titles,
                                  'Author1': a1,
                                  'Author2': a2,
                                  'Author3': a3,
                                  'Author4': a4,
                                  'Author5': a5,
                                  'Author6': a6,
                                  'Author7': a7,
                                  'Edition_Type': ed_type,
                                  'Edition_Month': ed_month,
                                  'Edition_Year': ed_year,
                                  'Ratings': ratings,
                                  'Reviews': reviews,
                                  'Genre1': g1,
                                  'Genre2': g2,
                                  'Category1': c1,
                                  'Category2': c2
                                  
                               })
    return structured_data

In [21]:
restructure(train).head(10)

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,THE PRISONER'S GOLD (THE HUNTERS 3),CHRIS KUZNESKI,None,None,None,None,None,None,PAPERBACK,MAR,2016,4.0,8,ACTION & ADVENTURE (BOOKS),None,ACTION & ADVENTURE,None
1,GURU DUTT: A TRAGEDY IN THREE ACTS,ARUN KHOPKAR,None,None,None,None,None,None,PAPERBACK,NOV,2012,3.9,14,CINEMA & BROADCAST (BOOKS),None,BIOGRAPHIES,DIARIES & TRUE ACCOUNTS
2,LEVIATHAN (PENGUIN CLASSICS),THOMAS HOBBES,None,None,None,None,None,None,PAPERBACK,FEB,1982,4.8,6,INTERNATIONAL RELATIONS,None,HUMOUR,None
3,A POCKET FULL OF RYE (MISS MARPLE),AGATHA CHRISTIE,None,None,None,None,None,None,PAPERBACK,OCT,2017,4.1,13,CONTEMPORARY FICTION (BOOKS),None,CRIME,THRILLER & MYSTERY
4,LIFE 70 YEARS OF EXTRAORDINARY PHOTOGRAPHY,EDITORS OF LIFE,None,None,None,None,None,None,HARDCOVER,OCT,2006,5.0,1,PHOTOGRAPHY TEXTBOOKS,None,ARTS,FILM & PHOTOGRAPHY
5,CHIRUNNING: A REVOLUTIONARY APPROACH TO EFFORT...,DANNY DREYER,None,None,None,None,None,None,PAPERBACK,MAY,2009,4.5,8,HEALTHY LIVING & WELLNESS (BOOKS),None,SPORTS,None
6,DEATH ON THE NILE (POIROT),AGATHA CHRISTIE,None,None,None,None,None,None,PAPERBACK,OCT,2017,4.4,72,CRIME,THRILLER & MYSTERY (BOOKS),CRIME,THRILLER & MYSTERY
7,YOGA YOUR HOME PRACTICE COMPANION: A COMPLETE ...,SIVANANDA YOGA VEDANTA CENTRE,None,None,None,None,None,None,HARDCOVER,MAR,2018,4.7,16,SPORTS TRAINING & COACHING (BOOKS),None,SPORTS,None
8,KARMAYOGI: A BIOGRAPHY OF E. SREEDHARAN,M S ASHOKAN,None,None,None,None,None,None,PAPERBACK,DEC,2015,4.2,111,BIOGRAPHIES & AUTOBIOGRAPHIES (BOOKS),None,BIOGRAPHIES,DIARIES & TRUE ACCOUNTS
9,"THE IRON KING (THE ACCURSED KINGS, BOOK 1)",MAURICE DRUON,None,None,None,None,None,None,PAPERBACK,MAR,2013,4.0,1,ACTION & ADVENTURE (BOOKS),None,ACTION & ADVENTURE,None


In [22]:
X_train = restructure(train)
Y_train = train['Price'].values
X_test = restructure(test)

In [23]:
X_train.describe(include="all")

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
count,6237,6237,6237,6237,6237,6237,6237,6237,6237,6237,6237.000000,6237.000000,6237.000000,6237,6237,6237,6237
unique,5564,3633,264,73,21,5,1,1,19,13,NaN,NaN,NaN,345,27,11,6
top,A GAME OF THRONES (A SONG OF ICE AND FIRE),AGATHA CHRISTIE,None,None,None,None,None,None,PAPERBACK,OCT,NaN,NaN,NaN,ACTION & ADVENTURE (BOOKS),None,ACTION & ADVENTURE,None
freq,4,69,5929,6159,6214,6233,6237,6237,5193,639,NaN,NaN,NaN,947,5594,818,3297
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.101972,4.293202,35.984287,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.821510,0.662501,149.995031,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000000,4.000000,2.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.000000,4.400000,7.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.000000,4.800000,22.000000,NaN,NaN,NaN,NaN


In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 17 columns):
Title            6237 non-null object
Author1          6237 non-null object
Author2          6237 non-null object
Author3          6237 non-null object
Author4          6237 non-null object
Author5          6237 non-null object
Author6          6237 non-null object
Author7          6237 non-null object
Edition_Type     6237 non-null object
Edition_Month    6237 non-null object
Edition_Year     6237 non-null int64
Ratings          6237 non-null float64
Reviews          6237 non-null int64
Genre1           6237 non-null object
Genre2           6237 non-null object
Category1        6237 non-null object
Category2        6237 non-null object
dtypes: float64(1), int64(2), object(14)
memory usage: 828.4+ KB


### Encoding Categorical Features

In [25]:
def unique_items(list1, list2):
    a = list1
    b = list2
    a.extend(b)
    return list(set(a))

In [26]:
from sklearn.preprocessing import LabelEncoder
le_title = LabelEncoder()
all_titles = unique_items(list(X_train.Title), list(X_test.Title))
le_title.fit(all_titles)

le_Editiontype = LabelEncoder()
all_edition = unique_items(list(X_train.Edition_Type), list(X_test.Edition_Type))
le_Editiontype.fit(all_edition)

le_EditionMonth = LabelEncoder()
all_month = unique_items(list(X_train.Edition_Month), list(X_test.Edition_Month))
le_EditionMonth.fit(all_month)

le_author = LabelEncoder()
all_Authors = list(set(all_authors))
le_author.fit(all_Authors)

le_genre = LabelEncoder()
all_Genres = list(set(genre_all))
le_genre.fit(all_Genres)

le_Category = LabelEncoder()
all_Categories = list(set(cat_all))
le_Category.fit(all_Categories)

LabelEncoder()

In [27]:
X_train['Title'] = le_title.transform(X_train['Title'])
X_train['Edition_Type'] = le_Editiontype.transform(X_train['Edition_Type'])
X_train['Edition_Month'] = le_EditionMonth.transform(X_train['Edition_Month'])

X_train['Author1'] = le_author.transform(X_train['Author1'])
X_train['Author2'] = le_author.transform(X_train['Author2'])
X_train['Author3'] = le_author.transform(X_train['Author3'])
X_train['Author4'] = le_author.transform(X_train['Author4'])
X_train['Author5'] = le_author.transform(X_train['Author5'])
X_train['Author6'] = le_author.transform(X_train['Author6'])
X_train['Author7'] = le_author.transform(X_train['Author7'])

X_train['Genre1'] = le_genre.transform(X_train['Genre1'])
X_train['Genre2'] = le_genre.transform(X_train['Genre2'])

X_train['Category1'] = le_Category.transform(X_train['Category1'])
X_train['Category2'] = le_Category.transform(X_train['Category2'])

In [28]:
X_train.head()

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5802,797,3090,3090,3090,3090,3090,3090,13,7,2016,4.0,8,0,268,0,12
1,2120,391,3090,3090,3090,3090,3090,3090,13,10,2012,3.9,14,80,268,2,6
2,2984,4354,3090,3090,3090,3090,3090,3090,13,3,1982,4.8,6,211,268,8,12
3,189,78,3090,3090,3090,3090,3090,3090,13,11,2017,4.1,13,98,268,5,16
4,2987,1221,3090,3090,3090,3090,3090,3090,8,11,2006,5.0,1,284,268,1,7


In [29]:
X_test['Title'] = le_title.transform(X_test['Title'])
X_test['Edition_Type'] = le_Editiontype.transform(X_test['Edition_Type'])
X_test['Edition_Month'] = le_EditionMonth.transform(X_test['Edition_Month'])

X_test['Author1'] = le_author.transform(X_test['Author1'])
X_test['Author2'] = le_author.transform(X_test['Author2'])
X_test['Author3'] = le_author.transform(X_test['Author3'])
X_test['Author4'] = le_author.transform(X_test['Author4'])
X_test['Author5'] = le_author.transform(X_test['Author5'])
X_test['Author6'] = le_author.transform(X_test['Author6'])
X_test['Author7'] = le_author.transform(X_test['Author7'])


X_test['Genre1'] = le_genre.transform(X_test['Genre1'])
X_test['Genre2'] = le_genre.transform(X_test['Genre2'])


X_test['Category1'] = le_Category.transform(X_test['Category1'])
X_test['Category2'] = le_Category.transform(X_test['Category2'])

In [30]:
X_test.head()

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5082,4059,3090,3090,3090,3090,3090,3090,12,11,1986,4.4,960,324,268,5,16
1,2906,1401,3090,3090,3090,3090,3090,3090,13,0,2018,5.0,1,273,268,4,9
2,751,949,3090,3090,3090,3090,3090,3090,13,7,2011,5.0,4,314,268,14,12
3,6232,169,3090,3090,3090,3090,3090,3090,13,9,2016,4.1,11,295,268,4,9
4,3790,3506,3090,3090,3090,3090,3090,3090,13,2,2011,4.4,9,235,268,10,11


### Normalizing the features

In [31]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

Y_train = Y_train.reshape(len(Y_train), 1)
Y_train = sc.fit_transform(Y_train)

# Restoring the original shape after scaling
Y_train = Y_train.ravel()

/home/abhishek/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/abhishek/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/abhishek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [32]:
X_train

array([[ 1.22489462, -1.11006858,  0.10538218, ..., -0.03756442,
        -1.21387465,  0.3167796 ],
       [-0.64983923, -1.40860357,  0.10538218, ..., -0.03756442,
        -0.82060991, -1.88135243],
       [-0.20992341,  1.50542146,  0.10538218, ..., -0.03756442,
         0.35918432,  0.3167796 ],
       ...,
       [ 0.91379674, -0.11299113,  0.10538218, ..., -0.03756442,
         1.53897855,  0.3167796 ],
       [-0.75676322, -1.56375352,  0.10538218, ..., -0.03756442,
        -1.21387465,  0.3167796 ],
       [ 0.95096556, -0.29534748,  0.10538218, ..., -0.03756442,
        -1.21387465,  0.3167796 ]])

In [33]:
Y_train

array([-0.49373942, -0.51847656, -0.37925584, ..., -0.23433993,
       -0.35027266, -0.15753449])

## Building a Regression Model

In [34]:
# creating and validating a dataset
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 123)

In [35]:
from xgboost import XGBRegressor 
xgb = XGBRegressor(max_depth=6, n_estimators=100, learning_rate=0.1, booster='gbtree', n_jobs=-1, random_state=1)
xgb.fit(train_x,train_y)

y_pred = sc.inverse_transform(xgb.predict(val_x))
y_val = sc.inverse_transform(val_y)

error = np.square(np.log10(y_pred+1) - np.log10(y_val+1)).mean() ** 0.5
score = 1 - error
print("RMLSE Score:", score)

RMLSE Score: 0.7164225998403633


In [36]:
# fitting the whole training set
xgb.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [37]:
# predicting for test set
preds = sc.inverse_transform(xgb.predict(X_test))

In [38]:
solution = pd.DataFrame(preds, columns=['Price'])
solution.to_excel('Soln_1.xlsx', index=False)

In [39]:
solution.head()

,Price
0,210.748322
1,1299.099365
2,627.625732
3,905.491943
4,437.703949


In [40]:
from IPython.display import FileLink, FileLinks
solution.to_excel("/home/abhishek/Documents/Soln_1.xlsx", index=False)
FileLinks('/home/abhishek/Documents/')

/home/abhishek/Documents/
  sentiment.py
  movie_recommender_sys.py
  Resume1.pdf
  Participants_Data-20190926T094854Z-001.zip
  Soln_1.xlsx
  test_pred.csv
  IJCIS-11-1-33-g002.png
  MLalgorithms-.pdf
  test_tweets_anuFYb8.csv
  test.csv
  My resume.pdf
  Resume.pdf
  train.csv
  train_E6oV3lV.csv
/home/abhishek/Documents/Participants_Data/
  Data_Train.xlsx
  Sample_Submission.xlsx
  Data_Test.xlsx

In [41]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [42]:
from bayes_opt import BayesianOptimization
import xgboost as xgb

In [43]:
Dtrain = xgb.DMatrix(X_train, label=Y_train)

In [50]:
def bo_tune_xgb(max_depth, gamma, n_estimators, learning_rate):
    params = {'max_depth':int(max_depth),
              'gamma':gamma,
              'n_estimators':int(n_estimators),
              'learning_rate':learning_rate,
              'subsample':0.8,
              'eta':0.1,
              'eval_metric':'rmse'
    }
    cv_result = xgb.cv(params, Dtrain, num_boost_round=100, nfold=10)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [51]:
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth':(1,300),
                                            'gamma':(0,1),
                                            'learning_rate':(0,1),
                                            'n_estimators':(1,1000)})
xgb_bo.maximize(n_iter=10, init_points=10 ,acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1222 extra nodes, 502 pruned nodes, max_depth=21
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1146 extra nodes, 488 pruned nodes, max_depth=23
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1166 extra nodes, 496 pruned nodes, max_depth=22
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1180 extra nodes, 482 pruned nodes, max_depth=21
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1234 extra nodes, 534 pruned nodes, max_depth=24
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1214 extra nodes, 542 pruned nodes, max_depth=24
[21:16:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1192 extra nodes, 522 pruned nodes, max_depth=22
[21:16:56] src/tree/updater_prune

[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 546 extra nodes, 3758 pruned nodes, max_depth=22
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 526 extra nodes, 3772 pruned nodes, max_depth=28
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 3774 pruned nodes, max_depth=28
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 554 extra nodes, 3776 pruned nodes, max_depth=25
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 3814 pruned nodes, max_depth=21
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 508 extra nodes, 3820 pruned nodes, max_depth=22
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 502 extra nodes, 3832 pruned nodes, max_depth=24
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 524 extra nodes, 3858 pruned nodes, max_depth=23
[21:16:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roo

[21:17:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra nodes, 4418 pruned nodes, max_depth=19
[21:17:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 4338 pruned nodes, max_depth=17
[21:17:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 4202 pruned nodes, max_depth=38
[21:17:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 4346 pruned nodes, max_depth=36
[21:17:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 4328 pruned nodes, max_depth=25
[21:17:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 4296 pruned nodes, max_depth=22
[21:17:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 228 extra nodes, 4260 pruned nodes, max_depth=34
[21:17:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 4424 pruned nodes, max_depth=28
[21:17:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roo

[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 4432 pruned nodes, max_depth=47
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 4394 pruned nodes, max_depth=26
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 4470 pruned nodes, max_depth=28
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 4472 pruned nodes, max_depth=20
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 4502 pruned nodes, max_depth=36
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 4394 pruned nodes, max_depth=20
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 4526 pruned nodes, max_depth=12
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 4462 pruned nodes, max_depth=32
[21:17:06] src/tree/updater_prune.cc:74: tree pruning end, 1 root

[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 4448 pruned nodes, max_depth=3
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 4440 pruned nodes, max_depth=18
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 4458 pruned nodes, max_depth=27
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 4450 pruned nodes, max_depth=13
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 4296 pruned nodes, max_depth=18
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 4476 pruned nodes, max_depth=35
[21:17:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 4402 pruned nodes, max_depth=24
[21:17:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 4402 pruned nodes, max_depth=27
[21:17:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 ext

...

[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 784 extra nodes, 962 pruned nodes, max_depth=20
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 748 extra nodes, 1008 pruned nodes, max_depth=19
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 760 extra nodes, 996 pruned nodes, max_depth=17
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 776 extra nodes, 942 pruned nodes, max_depth=18
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 722 extra nodes, 1024 pruned nodes, max_depth=20
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 682 extra nodes, 1028 pruned nodes, max_depth=18
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 756 extra nodes, 1012 pruned nodes, max_depth=18
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 740 extra nodes, 958 pruned nodes, max_depth=20
[21:30:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 746 extra nodes, 950 pruned nodes, max_depth=17
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 738 extra nodes, 1048 pruned nodes, max_depth=20
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 786 extra nodes, 1006 pruned nodes, max_depth=24
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 792 extra nodes, 950 pruned nodes, max_depth=20
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 744 extra nodes, 952 pruned nodes, max_depth=18
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 762 extra nodes, 976 pruned nodes, max_depth=19
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 750 extra nodes, 1040 pruned nodes, max_depth=19
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 734 extra nodes, 1022 pruned nodes, max_depth=20
[21:31:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 780 extra nodes, 944 pruned nodes, max_depth=19
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 760 extra nodes, 1016 pruned nodes, max_depth=20
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 748 extra nodes, 1016 pruned nodes, max_depth=20
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 718 extra nodes, 1026 pruned nodes, max_depth=19
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 752 extra nodes, 1010 pruned nodes, max_depth=20
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 748 extra nodes, 926 pruned nodes, max_depth=21
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 788 extra nodes, 956 pruned nodes, max_depth=22
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 752 extra nodes, 992 pruned nodes, max_depth=22
[21:31:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 800 extra nodes, 974 pruned nodes, max_depth=21
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 752 extra nodes, 954 pruned nodes, max_depth=21
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 752 extra nodes, 984 pruned nodes, max_depth=21
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 738 extra nodes, 1044 pruned nodes, max_depth=18
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 712 extra nodes, 930 pruned nodes, max_depth=20
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 778 extra nodes, 904 pruned nodes, max_depth=22
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 756 extra nodes, 922 pruned nodes, max_depth=19
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 720 extra nodes, 948 pruned nodes, max_depth=17
[21:31:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 754

[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 782 extra nodes, 964 pruned nodes, max_depth=19
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 760 extra nodes, 968 pruned nodes, max_depth=21
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 716 extra nodes, 992 pruned nodes, max_depth=21
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 768 extra nodes, 1036 pruned nodes, max_depth=19
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 784 extra nodes, 930 pruned nodes, max_depth=19
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 772 extra nodes, 926 pruned nodes, max_depth=21
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 704 extra nodes, 916 pruned nodes, max_depth=16
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 748 extra nodes, 990 pruned nodes, max_depth=18
[21:31:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 756

In [53]:
params = xgb_bo.max['params']
print(params)

params['max_depth']= int(round(params['max_depth']))
params['n_estimators']= int(round(params['n_estimators']))

print(params)

{'gamma': 0.3138771700338179, 'learning_rate': 0.12523151584542325, 'max_depth': 21.261063114462008, 'n_estimators': 756.8918737614696}
{'gamma': 0.3138771700338179, 'learning_rate': 0.12523151584542325, 'max_depth': 21, 'n_estimators': 757}


In [54]:
reg = XGBRegressor(**params).fit(X_train, Y_train)
y_pred_reg = sc.inverse_transform(reg.predict(X_test))

In [55]:
solution_bo = pd.DataFrame(y_pred_reg, columns=['Price'])
solution_bo.head()

,Price
0,199.287201
1,2040.762939
2,361.807159
3,1003.959229
4,365.029022


In [56]:
from IPython.display import FileLink, FileLinks
solution.to_excel("/home/abhishek/Documents/Soln_2.xlsx", index=False)
FileLinks('/home/abhishek/Documents/')

/home/abhishek/Documents/
  Soln_2.xlsx
  sentiment.py
  movie_recommender_sys.py
  Resume1.pdf
  Participants_Data-20190926T094854Z-001.zip
  Soln_1.xlsx
  test_pred.csv
  IJCIS-11-1-33-g002.png
  MLalgorithms-.pdf
  test_tweets_anuFYb8.csv
  test.csv
  My resume.pdf
  Resume.pdf
  train.csv
  train_E6oV3lV.csv
/home/abhishek/Documents/Participants_Data/
  Data_Train.xlsx
  Sample_Submission.xlsx
  Data_Test.xlsx